In [ ]:
import torch
import numpy as np
import pandas as pd

np.random.seed(314159) # set random seed

import torch
import pytorch_lightning as pl

from torch_geometric.data import Data

import wandb


In [ ]:
import numpy as np
import torch

# Load the edge list
edge_list = np.load("edge_list_latest1.npy", allow_pickle=True)

edge_list = np.array([row[0].split(",") for row in edge_list[1:]])

# Extract the source nodes, target nodes, and combined scores
protein1 = edge_list[:, 0] 
protein2 = edge_list[:, 1] 
combined_score = edge_list[:, 2].astype(np.float32)

# Mapping
protein_map = {protein: idx for idx, protein in enumerate(np.unique(np.concatenate((protein1, protein2))))}
protein1 = np.vectorize(protein_map.get)(protein1)
protein2 = np.vectorize(protein_map.get)(protein2)

# Stacking
edges = np.stack([protein1, protein2], axis=0)
edge_list = torch.tensor(edges, dtype=torch.long)  # Edge index tensor
edge_weight = torch.tensor(combined_score, dtype=torch.float32)  # Edge weight tensor

print("Edge list shape:", edge_list.shape)  # Should be [2, n_edges]
print("Edge weights shape:", edge_weight.shape)  # Should be [n_edges]
print("Protein map:", protein_map)  # Mapping of protein names to IDs

In [ ]:
data_path = '200_node_network_embeddings_latest.csv'# Network embeddings
node_dataset = pd.read_csv(data_path, index_col=0)
node_dataset

In [ ]:
xcell_path = "/home/user/Diamond_101225/diamond_rho/diamond_tpm/gene_xcell_spearman_rho_with_ENSG.csv"
print("Loading xCell PCs from:", xcell_path)
xcell = pd.read_csv(xcell_path, sep=None, engine='python')
if 'gene_id' in xcell.columns:
    gene_col = 'gene_id'
    xcell['ensembl'] = xcell['gene_id'].astype(str).str.split('.').str[0]
else:
    gene_col = xcell.columns[0]
    xcell['ensembl'] = xcell[gene_col].astype(str).str.split('.').str[0]

pc_cols = [c for c in xcell.columns if ('xcell' in str(c).lower() and 'pc' in str(c).lower()) or str(c).lower().startswith('xcell_pc')]

if len(pc_cols) < 1:
    possible = [c for c in xcell.columns if c not in (gene_col, 'Gene', 'ensembl')]
    pc_cols = possible[:20]

print("xCell PC columns (count={}): {}".format(len(pc_cols), pc_cols[:30]))

if len(pc_cols) == 0:
    print("no xCell PC columns detected.")

else:
    xcell_idx = xcell.set_index('ensembl')[pc_cols]

    # Alignment
    xcell_aligned = xcell_idx.reindex(node_dataset.index)

    missing_pct = xcell_aligned.isna().mean(axis=0).round(3) * 100
    print("Percent missing per xCell PC column:\n", missing_pct)

    fill_strategy = "mean" 
    if fill_strategy == "zero":
        xcell_aligned_filled = xcell_aligned.fillna(0.0)
    else:
        xcell_aligned_filled = xcell_aligned.fillna(xcell_aligned.mean())

    xcell_aligned_filled.columns = [f"xcell_{str(c)}" if not str(c).startswith("xcell_") else str(c) for c in xcell_aligned_filled.columns]

    # concatenation
    node_dataset = pd.concat([node_dataset, xcell_aligned_filled], axis=1)

    # savinh
    out_merge = "node_dataset_with_xcell_rho.csv"
    node_dataset.to_csv(out_merge)
    print("Saved merged node_dataset to:", out_merge)

In [ ]:
node_dataset.sort_index(inplace=True)
node_dataset.reset_index(drop=False, inplace=True)
assert((node_dataset.index.to_numpy()==np.arange(len(node_dataset))).all())

In [ ]:
label_name = 'my_label'

# find positives
pos_label_col = 'gda_score' 
node_dataset[pos_label_col].fillna(0, inplace=True) 
pos_labels = pd.array([1 if row[pos_label_col] else None for id_, row in node_dataset.iterrows()], dtype='Int32')
node_dataset[label_name] = pos_labels

def sample_negatives(PU_labels):
    '''randomly samples from the unlabeled samples'''
    num_pos = (PU_labels==1).sum()
    neg_inds = PU_labels[PU_labels.isna()].sample(num_pos).index

    return neg_inds

neg_label_inds = sample_negatives(node_dataset[label_name])
node_dataset.loc[neg_label_inds, label_name] = 0

node_dataset[label_name].value_counts()

In [ ]:
node_dataset.set_index(node_dataset.columns[0], inplace=True)

node_dataset

In [ ]:
label_col = node_dataset.columns[-1] 
node_dataset[label_col] = node_dataset[label_col].astype('Int32')

# Remove the 'gda_score' column
if 'gda_score' in node_dataset.columns:
    node_dataset = node_dataset.drop(columns=['gda_score'])

node_feat_cols = node_dataset.columns[:-1].tolist()  # All columns except the label


# Get subset of node features + labels
node_data = node_dataset[node_feat_cols + [label_col]]

# Convert features to PyTorch Tensor
X = torch.Tensor(node_data[node_feat_cols].select_dtypes(include=[np.number]).to_numpy(dtype=np.float32))


# Convert labels to PyTorch Tensor, filling NaN with -1
y = node_data[label_col].fillna(-1).astype('int')
y = torch.Tensor(y).type(torch.int64)

# Restrict to data with labels (non-NaN)
node_data_labeled = node_data[node_data[label_col].notna()]
node_data_labeled

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch

# Extract the labeled node indices and labels
X_myIDs = node_data_labeled.index.to_numpy() 
labels = node_data_labeled[label_col].to_numpy() 

test_size = 0.2
val_size = 0.1 * (1 / (1 - test_size)) 

# Perform stratified train-test split
myIDs_train_val, myIDs_test = train_test_split(X_myIDs, test_size=test_size, shuffle=True, stratify=labels)

# Perform stratified train-validation split
labels_train_val = node_data_labeled.loc[myIDs_train_val, label_col].to_numpy()
myIDs_train, myIDs_val = train_test_split(myIDs_train_val, test_size=val_size, shuffle=True, stratify=labels_train_val)

# Convert Ensembl IDs to Integer-Based Indices
# Create a mapping from Ensembl ID to row position
id_to_idx = {id_: idx for idx, id_ in enumerate(node_data.index)}

# Map train, val, and test IDs to their corresponding row indices
train_idx = np.array([id_to_idx[i] for i in myIDs_train])
val_idx = np.array([id_to_idx[i] for i in myIDs_val])
test_idx = np.array([id_to_idx[i] for i in myIDs_test])

# Create boolean masks
n_nodes = len(node_data)

train_mask = np.zeros(n_nodes, dtype=bool)
train_mask[train_idx] = True
train_mask = torch.tensor(train_mask, dtype=torch.bool)

val_mask = np.zeros(n_nodes, dtype=bool)
val_mask[val_idx] = True
val_mask = torch.tensor(val_mask, dtype=torch.bool)

test_mask = np.zeros(n_nodes, dtype=bool)
test_mask[test_idx] = True
test_mask = torch.tensor(test_mask, dtype=torch.bool)

# Print distributions
print(f"Number of training nodes: {train_mask.sum().item()}")
print(f"Number of validation nodes: {val_mask.sum().item()}")
print(f"Number of test nodes: {test_mask.sum().item()}")


In [ ]:
data = Data(x=X, y=y, edge_index=edge_list, edge_attr=edge_weight)  # Add edge weights

num_classes = 2
num_features = X.shape[1]

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
assert y.shape[0] == X.shape[0], "Mismatch: y and X must have the same number of nodes"


print(data)

In [ ]:
import wandb
import os

os.environ["WANDB_API_KEY"] = "" #WANDB API KEY
os.environ["WANDB_MODE"] = "online"

config = {
    "dataset": "CIFAR10",
    "machine": "online cluster",
    "model": "CNN",
    "learning_rate": 0.01,
    "batch_size": 128,
}

wandb.init(project="offline-demo")

for i in range(100):
    wandb.log({"accuracy": i})

In [ ]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

from torch_geometric.nn import MessagePassing
from torch_geometric.data import DataLoader

from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    ModelSummary,
    EarlyStopping
)
from pytorch_lightning.loggers import WandbLogger

import datetime
class WeightedSAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr="mean")

        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.lin_update = torch.nn.Linear(in_channels + out_channels, out_channels)

        self.last_messages = None 

    def forward(self, x, edge_index, edge_weight):
        x_transformed = self.lin(x)

        out = self.propagate(
            edge_index=edge_index,
            x=x_transformed,
            edge_weight=edge_weight
        )

        out = self.lin_update(torch.cat([out, x], dim=1))
        return out

    def message(self, x_j, edge_weight):
        messages = edge_weight.view(-1, 1) * x_j
        self.last_messages = messages.detach().cpu()
        return messages
class GNNModel(torch.nn.Module):
    def __init__(
        self,
        num_features,
        hidden_channels,
        num_classes,
        hidden_dense,
        GNN_conv_layer,
        dropout_rate=0.3,
    ):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GNN_conv_layer(num_features, hidden_channels[0]))

        for c1, c2 in zip(hidden_channels[:-1], hidden_channels[1:]):
            self.convs.append(GNN_conv_layer(c1, c2))

        self.dense1 = torch.nn.Linear(hidden_channels[-1], hidden_dense)
        self.dense_out = torch.nn.Linear(hidden_dense, num_classes)

        self.dropout_rate = dropout_rate

    def forward(self, x, edge_index, edge_weight):
        for conv in self.convs:
            x = conv(x, edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = self.dense1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.dense_out(x)

        return x
class LitGNN(pl.LightningModule):
    def __init__(
        self,
        model_name,
        num_features,
        hidden_channels,
        num_classes,
        hidden_dense,
        GNN_conv_layer,
        dropout_rate,
        lr=1e-3,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.model = GNNModel(
            num_features=num_features,
            hidden_channels=hidden_channels,
            num_classes=num_classes,
            hidden_dense=hidden_dense,
            GNN_conv_layer=GNN_conv_layer,
            dropout_rate=dropout_rate,
        )

        self.criterion = torch.nn.CrossEntropyLoss()
        self.validation_step_outputs = []

    def forward(self, batch, x_override=None):
        x = x_override if x_override is not None else batch.x
        return self.model(x, batch.edge_index, batch.edge_attr)

    def training_step(self, batch, batch_idx):
        out = self(batch)
        valid = batch.y >= 0

        loss = (
            self.criterion(out[valid], batch.y[valid])
            if valid.any()
            else torch.tensor(0.0, device=self.device, requires_grad=True)
        )

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        out = self(batch)
        valid = batch.y >= 0

        if valid.any():
            loss = self.criterion(out[valid], batch.y[valid])
            acc = (out[valid].argmax(dim=1) == batch.y[valid]).float().mean()
        else:
            loss = torch.tensor(0.0, device=self.device)
            acc = torch.tensor(0.0, device=self.device)

        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)

        self.validation_step_outputs.append(acc)

    def on_validation_epoch_end(self):
        if self.validation_step_outputs:
            mean_acc = torch.stack(self.validation_step_outputs).mean()
            self.log("val_acc_epoch", mean_acc, prog_bar=True)
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        out = self(batch)
        valid = batch.y >= 0

        if valid.any():
            loss = self.criterion(out[valid], batch.y[valid])
            acc = (out[valid].argmax(dim=1) == batch.y[valid]).float().mean()
        else:
            loss = torch.tensor(0.0, device=self.device)
            acc = torch.tensor(0.0, device=self.device)

        self.log("test_loss", loss, on_epoch=True)
        self.log("test_acc", acc, on_epoch=True)
        return acc

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl
import torch
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
import datetime

AVAIL_GPUS = 0
MAX_EPOCHS = 200
NUM_TRIALS = 25

GNN_conv_layer = WeightedSAGEConv

for trial in range(1, NUM_TRIALS + 1):

    model_name = f"graphsage_{datetime.datetime.today().strftime('%Y-%m-%d')}_trial_{trial}"

    logger = WandbLogger(
        name=model_name,
        project="",
        log_model="all",
    )

    model = LitGNN(
        model_name="GraphSAGE",
        num_features=num_features,
        hidden_channels=[128],
        num_classes=num_classes,
        hidden_dense=64,
        GNN_conv_layer=GNN_conv_layer,
        dropout_rate=0.3,
    )

    train_loader = DataLoader([data], batch_size=1, num_workers=0)
    val_loader = DataLoader([data], batch_size=1, num_workers=0)

    checkpoint_callback = ModelCheckpoint(
        save_weights_only=True,
        monitor="val_acc",
        mode="max",
        dirpath=f"checkpoints/trial_{trial}",
        filename="{epoch:02d}-{val_acc:.3f}",
    )

    trainer = pl.Trainer(
        accelerator="cpu",
        devices=1,
        max_epochs=MAX_EPOCHS,
        logger=logger,
        callbacks=[
            checkpoint_callback,
            EarlyStopping(monitor="val_acc", patience=20, mode="max"),
            ModelSummary(max_depth=3),
        ],
    )

    trainer.fit(model, train_loader, val_loader)

    if checkpoint_callback.best_model_path:
        model = LitGNN.load_from_checkpoint(
            checkpoint_callback.best_model_path,
            model_name="GraphSAGE",
            num_features=num_features,
            hidden_channels=[128],
            num_classes=num_classes,
            hidden_dense=64,
            GNN_conv_layer=GNN_conv_layer,
            dropout_rate=0.3,
        )

        print(f"Trial {trial}: loaded {checkpoint_callback.best_model_path}")
    else:
        print(f"Trial {trial}: no checkpoint found")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

train_accuracies, train_precisions, train_recalls, train_f1s = [], [], [], []
test_accuracies, test_precisions, test_recalls, test_f1s = [], [], [], []

for trial in range(1, NUM_TRIALS + 1):
    print(f"\nEvaluating Trial {trial}")

    data = data.to(device='cpu') 

    logits = model(data)
    preds_train = logits[data.train_mask].argmax(dim=-1)
    preds_test = logits[data.test_mask].argmax(dim=-1)

    y_train = data.y[data.train_mask]
    y_test = data.y[data.test_mask]

    train_report = classification_report(
        y_train.cpu(), preds_train.cpu(), labels=[0, 1], target_names=['negative', 'positive'], output_dict=True
    )
    test_report = classification_report(
        y_test.cpu(), preds_test.cpu(), labels=[0, 1], target_names=['negative', 'positive'], output_dict=True
    )

    train_accuracies.append(train_report['accuracy'])
    train_precisions.append(train_report['positive']['precision'])
    train_recalls.append(train_report['positive']['recall'])
    train_f1s.append(train_report['positive']['f1-score'])
    
    test_accuracies.append(test_report['accuracy'])
    test_precisions.append(test_report['positive']['precision'])
    test_recalls.append(test_report['positive']['recall'])
    test_f1s.append(test_report['positive']['f1-score'])

    print(f"Trial {trial} Training Metrics:\n{train_report}")
    print(f"Trial {trial} Testing Metrics:\n{test_report}")

print("\nAggregated Metrics (Mean ± Std):")
print(f"Training Accuracy: {np.mean(train_accuracies):.4f} ± {np.std(train_accuracies):.4f}")
print(f"Training Precision: {np.mean(train_precisions):.4f} ± {np.std(train_precisions):.4f}")
print(f"Training Recall: {np.mean(train_recalls):.4f} ± {np.std(train_recalls):.4f}")
print(f"Training F1-Score: {np.mean(train_f1s):.4f} ± {np.std(train_f1s):.4f}")
print(f"Testing Accuracy: {np.mean(test_accuracies):.4f} ± {np.std(test_accuracies):.4f}")
print(f"Testing Precision: {np.mean(test_precisions):.4f} ± {np.std(test_precisions):.4f}")
print(f"Testing Recall: {np.mean(test_recalls):.4f} ± {np.std(test_recalls):.4f}")
print(f"Testing F1-Score: {np.mean(test_f1s):.4f} ± {np.std(test_f1s):.4f}")
